In [ ]:
#import pyddm
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from time import time
from mpl_toolkits.mplot3d import Axes3D
import scipy.optimize
import pandas as pd
from scipy.stats import gaussian_kde
from scipy.optimize import minimize



def simulation_ddm(num_simulations, time_steps,drift_rate, boundary_separation, starting_point, noise_std, dt):
  # Arrays to store results
  decision_times1 = []
  decision_times2 = []
  no_decision = []

  for i in range(num_simulations):
      # Initialize decision variable
      decision_variable = starting_point
      for t in range(time_steps):
          # Update decision variable with drift and noise
          noise = np.random.normal(0, noise_std)
          decision_variable += drift_rate * dt + noise * np.sqrt(dt)

          # Check if decision boundary is crossed
          if decision_variable >= boundary_separation:
              decision_times1.append(t * dt)
              break
          elif decision_variable <= -boundary_separation:
              decision_times2.append(t * dt)
              break
      else:
          # If no decision is made within time_steps, record maximum time
          no_decision.append(time_steps * dt)
  return decision_times1,decision_times2


def simulation_ddm_rk4(num_simulations, time_steps, drift_rate, boundary_separation, starting_point, noise_std, dt):
    # Arrays to store results
    decision_times1 = []
    decision_times2 = []
    no_decision = []

    # Generate noise once for efficiency
    noise = np.random.normal(0, noise_std, (num_simulations, time_steps))

    for i in range(num_simulations):
        # Initialize decision variable
        decision_variable = starting_point
        for t in range(time_steps):
            # RK4 method: compute four slopes (k1, k2, k3, k4)
            k1 = drift_rate * dt + noise[i, t] * np.sqrt(dt)
            k2 = drift_rate * dt + noise[i, t] * np.sqrt(dt)
            k3 = drift_rate * dt + noise[i, t] * np.sqrt(dt)
            k4 = drift_rate * dt + noise[i, t] * np.sqrt(dt)

            # Update decision variable using the RK4 formula
            decision_variable += (k1 + 2 * k2 + 2 * k3 + k4) / 6

            # Check if decision boundary is crossed
            if decision_variable >= boundary_separation:
                decision_times1.append(t * dt)  # Time of first boundary crossing
                break
            elif decision_variable <= -boundary_separation:
                decision_times2.append(t * dt)  # Time of second boundary crossing
                break
        else:
            # If no decision is made within time_steps, record maximum time
            no_decision.append(time_steps * dt)

    # Return results
    return np.array(decision_times1), np.array(decision_times2)



# A sample
# Simulation parameters
num_simulations = 4000  # Number of simulated decision processes
time_steps = 20000       # Number of time steps per simulation
dt = 0.01               # Time step size
drift_rate = .5       # Drift rate (v)
boundary_separation = 2  # Boundary separation (a)
starting_point = 0.0     # Starting point (z)
noise_std = 1.     # Standard deviation of noise (σ)

decision_times1,decision_times2 = simulation_ddm_rk4(num_simulations, time_steps,drift_rate, boundary_separation, starting_point, noise_std, dt)
rt1 = np.sort(decision_times1)
rt2 = np.sort(decision_times2)

kde1 = gaussian_kde(rt1)
plt.plot(rt1,len(rt1)*kde1(rt1)[:, np.newaxis]/(len(rt1)+len(rt2)))

kde2 = gaussian_kde(rt2)
plt.plot(rt2,-len(rt2)*kde2(rt2)[:, np.newaxis]/(len(rt1)+len(rt2)))




# Define model architecture
class PINN_NeuralNet(tf.keras.Model):
    """ Set basic architecture of the PINN model."""

    def __init__(self, lb, ub,
            output_dim=1,
            num_hidden_layers=5,
            num_neurons_per_layer=50,
            activation='tanh',
            kernel_initializer='glorot_normal',
            **kwargs):
        super().__init__(**kwargs)

        self.num_hidden_layers = num_hidden_layers
        self.output_dim = output_dim
        self.lb = lb
        self.ub = ub
        ## Define NN architecture
        self.hidden = [tf.keras.layers.Dense(num_neurons_per_layer,
                             activation=tf.keras.activations.get(activation),
                             kernel_initializer=kernel_initializer)
                           for _ in range(self.num_hidden_layers-1)]
        self.hidden1 = tf.keras.layers.Dense(num_neurons_per_layer,activation = 'softplus')
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, X):
        """Forward-pass through neural network."""
        Z = X
        for i in range(self.num_hidden_layers-1):
            Z = self.hidden[i](Z)
        #Z = self.hidden1(Z)
        return self.out(Z)

class PINNIdentificationNet(PINN_NeuralNet):
    def __init__(self, *args, **kwargs):

        # Call init of base class
        super().__init__(*args,**kwargs)

        # Initialize variable for lambda
        #self.lambd = tf.Variable(0.0, trainable=True, dtype='float32')
        #self.sig = tf.Variable(2.0, trainable=True, dtype='float32')
        #self.tt0 = tf.Variable(0.0, trainable=True, dtype='float32')
        self.lambd = self.add_weight(
            name="lambd_raw", initializer="ones",trainable=True, dtype=tf.float32)
        self.sig = self.add_weight(name="sig", initializer="ones" ,trainable=True, dtype=tf.float32)
        self.tt0 = self.add_weight(name="tt0",initializer="ones",trainable=True, dtype=tf.float32)

        self.lambd_list = []
        self.sig_list = []
        self.tt0_list = []


class PINNSolver():
    def __init__(self, model, X_r):
        self.model = model

        # Store collocation points
        self.t = X_r[:,0:1]
        self.x = X_r[:,1:2]

        # Initialize history of losses and global iteration counter
        self.hist = []
        self.iter = 0

    def get_r(self):

        with tf.GradientTape(persistent=False) as tape:
            # Watch variables representing t and x during this GradientTape
            tape.watch(self.t)
            tape.watch(self.x)

            # Compute current values u(t,x)
            u = self.model(tf.stack([self.t[:,0], self.x[:,0]], axis=1))

            u_x = tape.gradient(u, self.x)

        u_t = tape.gradient(u, self.t)
        u_xx = tape.gradient(u_x, self.x)

        del tape
        return self.fun_r(self.t, self.x, u, u_t, u_x, u_xx)

    def loss_fn(self, X,xmax,rt1,rt2, u):
        # Compute phi_r
        r= self.get_r()
        phi_r = tf.reduce_mean(tf.square(r))

        # Initialize loss
        loss_r = phi_r

        # Add phi_0 and phi_b to the loss

        u_pred = self.model(X[0])
        loss_0 = tf.reduce_mean(tf.square(u[0] - u_pred))
        #loss_0 = tf.reduce_mean(tf.square(tfp.math.trapz(u_pred, x=X[0][:,1:2])-1))

        u_pred = self.model(X[1])
        loss_b = tf.reduce_mean(tf.square(u[1] - u_pred))

        tspace1 = np.sort(rt1)
        tspace2 = np.sort(rt2)
        tspace_tf1 = tf.constant(tspace1.reshape((len(rt1),1)),'float32')
        minRT1 = min(rt1)
        minRT2 = min(rt2)
        self.minRT = min(minRT1,minRT2)
        t1 = tspace_tf1 - tf.math.sigmoid(self.model.tt0)*self.minRT


        xspace = np.ones((len(tspace1),1))*xmax
        xtf = tf.constant(xspace,'float32')
        X = tf.concat([t1,xtf],1)
        p_i =  self.model(X)

        xspace1 = np.ones((len(tspace1),1))*(xmax-0.02)
        xtf1 = tf.constant(xspace1,'float32')
        X1 = tf.concat([t1,xtf1],1)
        p_ii =  self.model(X1)

        xspace2 = np.ones((len(tspace1),1))*(xmax-0.04)
        xtf2 = tf.constant(xspace2,'float32')
        X2 = tf.concat([t1,xtf2],1)
        p_iii =  self.model(X2)
        p_x = (3*p_i-4*p_ii+p_iii)/(2*0.02)

        J1 = tf.nn.softplus(self.model.lambd)* p_i -0.5*(self.model.sig)**2*p_x
        kde1 = gaussian_kde(tspace1)
        p_kde1 = len(tspace1)*kde1(tspace1)[:, np.newaxis]/(len(tspace2)+len(tspace1))
        p_kde_tensor1 = tf.convert_to_tensor(p_kde1, dtype=tf.float32)
        KDE_loss1 = tf.reduce_mean(tf.square(J1 - p_kde_tensor1))

#####################################################3
        tspace_tf2 = tf.constant(tspace2.reshape((len(rt2),1)),'float32')

        t2 = tspace_tf2 - tf.math.sigmoid(self.model.tt0)*self.minRT


        xspace20 = np.ones((len(tspace2),1))*(-xmax)
        xtf20 = tf.constant(xspace20,'float32')
        X20 = tf.concat([t2,xtf20],1)
        p_i2 =  self.model(X20)

        xspace21 = np.ones((len(tspace2),1))*(-xmax+0.02)
        xtf21 = tf.constant(xspace21,'float32')
        X21 = tf.concat([t2,xtf21],1)
        p_ii2 =  self.model(X21)

        xspace22 = np.ones((len(tspace2),1))*(-xmax+0.04)
        xtf22 = tf.constant(xspace22,'float32')
        X22 = tf.concat([t2,xtf22],1)
        p_iii2 =  self.model(X22)
        p_x2 = (-3*p_i2+4*p_ii2 -p_iii2)/(2*0.02)

        J2 = tf.nn.softplus(self.model.lambd)* p_i2 -0.5*(self.model.sig)**2*p_x2
        kde2 = gaussian_kde(tspace2)
        p_kde2 =- len(tspace2)*kde2(tspace2)[:, np.newaxis]/(len(tspace2)+len(tspace1))
        p_kde_tensor2 = tf.convert_to_tensor(p_kde2, dtype=tf.float32)
        KDE_loss2 = tf.reduce_mean(tf.square(J2 - p_kde_tensor2))

        return 100*(loss_r+loss_0+loss_b) + KDE_loss1 + KDE_loss2


    def get_grad(self, X,xmax,rt1,rt2, u):
        with tf.GradientTape() as tape:
            # This tape is for derivatives with
            # respect to trainable variables
            #tape.watch(self.model.trainable_variables)

            loss = self.loss_fn(X,xmax,rt1,rt2, u)

        g = tape.gradient(loss, self.model.trainable_variables)
        del tape
        return loss, g

    #def fun_r(self, t, x, u, u_t, u_x, u_xx):
        """Residual of the PDE"""
     #   return u_t + tf.nn.softplus(self.model.lambd)*u_x - 0.5 *(tf.nn.softplus(self.model.sig))**2* u_xx

    def solve_with_TFoptimizer(self, optimizer, X,xmax,rt1,rt2, u, N=1001):
        """This method performs a gradient descent type optimization."""

        @tf.function
        def train_step():
            loss, grad_theta = self.get_grad(X,xmax,rt1,rt2, u)

            # Perform gradient descent step
            optimizer.apply_gradients(zip(grad_theta, self.model.trainable_variables))
            return loss

        for i in range(N):

            loss = train_step()

            self.current_loss = loss.numpy()
            self.callback()

    def solve_with_ScipyOptimizer(self, X,rt, u, method='L-BFGS-B', **kwargs):
        """This method provides an interface to solve the learning problem
        using a routine from scipy.optimize.minimize.
        (Tensorflow 1.xx had an interface implemented, which is not longer
        supported in Tensorflow 2.xx.)
        Type conversion is necessary since scipy-routines are written in Fortran
        which requires 64-bit floats instead of 32-bit floats."""

        def get_weight_tensor():
            """Function to return current variables of the model
            as 1d tensor as well as corresponding shapes as lists."""

            weight_list = []
            shape_list = []

            # Loop over all variables, i.e. weight matrices, bias vectors and unknown parameters
            for v in self.model.variables:
                shape_list.append(v.shape)
                weight_list.extend(v.numpy().flatten())

            weight_list = tf.convert_to_tensor(weight_list)
            return weight_list, shape_list

        x0, shape_list = get_weight_tensor()

        def set_weight_tensor(weight_list):
            """Function which sets list of weights
            to variables in the model."""
            idx = 0
            for v in self.model.variables:
                vs = v.shape

                # Weight matrices
                if len(vs) == 2:
                    sw = vs[0]*vs[1]
                    new_val = tf.reshape(weight_list[idx:idx+sw],(vs[0],vs[1]))
                    idx += sw

                # Bias vectors
                elif len(vs) == 1:
                    new_val = weight_list[idx:idx+vs[0]]
                    idx += vs[0]

                # Variables (in case of parameter identification setting)
                elif len(vs) == 0:
                    new_val = weight_list[idx]
                    idx += 1

                # Assign variables (Casting necessary since scipy requires float64 type)
                v.assign(tf.cast(new_val, 'float32'))

        def get_loss_and_grad(w):
            """Function that provides current loss and gradient
            w.r.t the trainable variables as vector. This is mandatory
            for the LBFGS minimizer from scipy."""

            # Update weights in model
            set_weight_tensor(w)
            # Determine value of \phi and gradient w.r.t. \theta at w
            loss, grad = self.get_grad(X,rt, u)

            # Store current loss for callback function
            loss = loss.numpy().astype(np.float64)
            self.current_loss = loss

            # Flatten gradient
            grad_flat = []
            for g in grad:
                grad_flat.extend(g.numpy().flatten())

            # Gradient list to array
            grad_flat = np.array(grad_flat,dtype=np.float64)

            # Return value and gradient of \phi as tuple
            return loss, grad_flat


        return scipy.optimize.minimize(fun=get_loss_and_grad,
                                       x0=x0,
                                       jac=True,
                                       method=method,
                                       callback=self.callback,
                                       **kwargs)

    def callback(self, xr=None):
        if self.iter % 50 == 0:
            print('It {:05d}: loss = {:10.8e}'.format(self.iter,self.current_loss))
        self.hist.append(self.current_loss)
        self.iter+=1


    def plot_solution(self, **kwargs):
        N = 600
        tspace = np.linspace(self.model.lb[0], self.model.ub[0], N+1)
        xspace = np.linspace(self.model.lb[1], self.model.ub[1], N+1)
        T, X = np.meshgrid(tspace, xspace)
        Xgrid = np.vstack([T.flatten(),X.flatten()]).T
        upred = self.model(tf.cast(Xgrid,'float32'))
        U = upred.numpy().reshape(N+1,N+1)
        fig = plt.figure(figsize=(9,6))
        ax = fig.add_subplot(111, projection='3d')
        ax.plot_surface(T, X, U, cmap='viridis', **kwargs)
        ax.set_xlabel('$t$')
        ax.set_ylabel('$x$')
        ax.set_zlabel('$u_\\theta(t,x)$')
        ax.view_init(35,35)
        return ax

    def plot_loss_history(self, ax=None):
        if not ax:
            fig = plt.figure(figsize=(7,5))
            ax = fig.add_subplot(111)
        ax.semilogy(range(len(self.hist)), self.hist,'k-')
        ax.set_xlabel('$n_{epoch}$')
        ax.set_ylabel('$\\phi^{n_{epoch}}$')
        return ax


class F_P_PINNSolver(PINNSolver):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_r(self):

        with tf.GradientTape(persistent=True) as tape:
            # Watch variables representing t and x during this GradientTape
            tape.watch(self.t)
            tape.watch(self.x)

            # Compute current values u(t,x)
            u = self.model(tf.stack([self.t[:,0], self.x[:,0]], axis=1))

            u_x = tape.gradient(u, self.x)

        u_t = tape.gradient(u, self.t)
        u_xx = tape.gradient(u_x, self.x)
        del tape

        return self.fun_r(self.t, self.x, u, u_t, u_x,u_xx)

class F_P_PINNIdentification(F_P_PINNSolver):

    def fun_r(self, t, x, u, u_t, u_x, u_xx):
        """Residual of the PDE"""
        # Define residual of the PDE
        return u_t + tf.nn.softplus(self.model.lambd)*u_x - 0.5 *(self.model.sig)**2* u_xx
        #return u_t + self.model.lambd*u_x - 0.5 *(self.model.sig)**2* u_xx

    def callback(self, xr=None):
        mu = tf.nn.softplus(self.model.lambd)
        sigma= self.model.sig
        lambd = mu.numpy()
        sig =sigma.numpy()
        tt0 =(tf.math.sigmoid(self.model.tt0)*self.minRT).numpy()
        #lambd = self.model.lambd.numpy()
        self.model.lambd_list.append(lambd)
        self.model.sig_list.append(sig)
        self.model.tt0_list.append(tt0)
        if self.iter % 50 == 0:
            print('It {:05d}: loss = {:10.8e} drift = {:10.8e}  sigma = {:10.8e}  t0 = {:10.8e}'.format(self.iter, self.current_loss, lambd,sig, tt0-0.1))

        self.hist.append(self.current_loss)
        self.iter += 1

    def plot_loss_and_param(self, axs=None):
        if axs:
            ax1, ax2 = axs
            self.plot_loss_history(ax1)
        else:
            ax1 = self.plot_loss_history()
            ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

        color = 'tab:blue'
        ax2.tick_params(axis='y', labelcolor=color)
        ax2.plot(range(len(self.hist)), self.model.lambd_list,'-',color=color)
        ax2.set_ylabel('$\\lambda^{n_{epoch}}$', color=color)
        return (ax1,ax2)
    def lambdaf(self):
      mu = tf.nn.softplus(self.model.lambd)
      return mu.numpy()
            #return self.model.lambd.numpy()

    def sigmaf(self):
        sigma= self.model.sig
        return sigma.numpy()

    def tt0f(self):
            return (tf.math.sigmoid(self.model.tt0)*self.minRT).numpy()

num_simulations = 2000  # Number of simulated decision processes
time_steps = 20000       # Number of time steps per simulation
dt = 0.01               # Time step size
drift_rate = 0.5  # Drift rate (v)
boundary_separation = 2.0  # Boundary separation (a)
starting_point = 0.0     # Starting point (z)
noise_std = 1   # Standard deviation of noise (σ)
non_Time = 0.3
decision_times1,decision_times2 = simulation_ddm_rk4(num_simulations, time_steps,drift_rate, boundary_separation, starting_point, noise_std, dt)
rt1 = np.sort(decision_times1) + non_Time
rt2 = np.sort(decision_times2) + non_Time


RT1 = rt1
RT2 = rt2
threshold = 2
epoch = 50000
# Mesh Grid
N = 100
xmax = threshold
#xmax = threshold
max1 = max(RT1)
xmin = -xmax
lb = [0, xmin]
maxrt1 = max(RT1)
maxrt2 = max(RT2)
ub = [max(maxrt1,maxrt2),xmax]
tspace = np.linspace(lb[0], ub[0], N + 1)
xspace = np.linspace(lb[1], ub[1], N + 1)
T, X = np.meshgrid(tspace, xspace)
Xgrid = np.vstack([T.flatten(),X.flatten()]).T
#tspace = np.random.uniform(lb[0], ub[0],N)
#xspace = np.random.uniform(lb[1], ub[1],N)
#Xgrid = np.vstack([tspace,xspace]).T
Xgrid = tf.constant(Xgrid,'float32')




# Boundary data
N_b = 500
t_b = tf.random.uniform((N_b,1), lb[0], ub[0], dtype='float32')
x_b = lb[1] + (ub[1] - lb[1]) * tf.keras.backend.random_bernoulli((N_b,1), 0.5, dtype='float32')
X_b = tf.concat([t_b, x_b], axis=1)
X_b
u_b = tf.zeros(tf.shape(x_b),'float32')


delta = 7.8*10**(-2)

# direct’s delta function
def ddf(x,delta,x0):
    return 1/(2*np.sqrt(np.pi*delta))*tf.math.exp(-((x-x0)**2)/(4*delta))

# Define initial condition
def fun_u_0(x):
    return ddf(x,delta,0)


N_0 = 50
t_0 = tf.ones((N_0,1), dtype='float32')*lb[0]    #t = 0
x_0 = np.linspace(lb[1], ub[1], N_0-1, dtype='float32')
x_0 = np.asarray(list(x_0) + [0.0])
x_0 = np.sort(x_0)
#u_0 = np.where(x_0==0,1,0)
#uu_0 = 2.5*smooth(u_0, 1)
x_0 = tf.convert_to_tensor(x_0,dtype='float32')
x_0 = tf.reshape(x_0 ,[N_0,1])
u_0 = fun_u_0(x_0)
#plt.plot(x_0,u_0)

X_0 = tf.concat([t_0, x_0], axis=1)

X_param = [X_0,X_b]
u_param = [u_0, u_b]

# dtype in Tensorflow
lb = tf.constant([0, xmin],dtype='float32')
ub = tf.constant([max(maxrt1,maxrt2),xmax],dtype='float32')

# Initialize model
model = PINNIdentificationNet(lb, ub, num_hidden_layers=4,num_neurons_per_layer=30,
                                        activation='tanh',kernel_initializer='glorot_normal')
model.build(input_shape=(None,2))
# Initilize solver
f_p_Identification = F_P_PINNIdentification(model, Xgrid)

# Choose step sizes aka learning rate
lr =  tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,15000],[0.01,0.001,0.0005])

# Solve with Adam optimizer
optim = tf.keras.optimizers.Adam(learning_rate=lr) ####################################

# Start timer
t0 = time()
f_p_Identification.solve_with_TFoptimizer(optim, X_param ,xmax,RT1,RT2, u_param, N=epoch)
print('\nComputation time: {} seconds'.format(time()-t0))
#f_p_Identification.plot_solution()
#f_p_Identification.plot_loss_and_param()
print(drift_rate,noise_std,non_Time)
print(f_p_Identification.lambdaf(),f_p_Identification.sigmaf(),f_p_Identification.tt0f()-0.1)



M = 500
DTYPE='float32'
tspace = np.linspace(0, ub[0], M + 1)
xspace = np.ones((M+1))*ub[1]
X = np.zeros((M+1,2))
X[:,0] = tspace
X[:,1] = xspace
X = tf.constant(X,DTYPE)
p_i =  model(X)

xspace1 = np.ones((M+1))*(ub[1]-0.02)
X1 = np.zeros((M+1,2))
X1[:,0] = tspace
X1[:,1] = xspace1
X1 = tf.constant(X1,DTYPE)
p_ii =  model(X1)

xspace2 = np.ones((M+1))*(ub[1]-0.04)
X2 = np.zeros((M+1,2))
X2[:,0] = tspace
X2[:,1] = xspace2
X2 = tf.constant(X2,DTYPE)
p_iii =  model(X2)
p_x = (3*p_i-4*p_ii+p_iii)/(2*0.02)
J1 = f_p_Identification.lambdaf()* p_i -0.5*f_p_Identification.sigmaf()**2*p_x



def f(v,a,t):
      return (a/np.sqrt(2*np.pi*t**3)*np.exp(-(v*t-a)**2/(2*t)))
F1 = []
C = []
for rt in np.sort(tspace):
  F1.append(f(drift_rate,boundary_separation,rt))
  C.append(np.trapz(F1))

kde1 = gaussian_kde(rt1)
kde2 = gaussian_kde(rt2)

import matplotlib.pyplot as plt
import numpy as np

# قالب فونت یکسان
font = {'family': 'serif',
        'color':  'black',
        'weight': 'normal',
        'size': 16,
        }


plt.figure(figsize=(12, 8), dpi=90)


plt.plot(tspace + 0.07, J1, label='Approximated', color='black')
plt.plot(tspace, np.asarray(F1), label='Exact', color='black', linestyle='--')
plt.plot(rt1 - 0.3, len(rt1) * kde1(rt1)[:, np.newaxis] / (len(rt1) + len(rt2)),
         label='Data', color='red', linestyle='-.')

plt.xlabel('Time', fontdict=font)
plt.ylabel('First passage time', fontdict=font)


plt.legend(fontsize=18)


plt.tick_params(axis='both', which='major', labelsize=14)


plt.savefig("ex1-10.pdf", format="pdf", bbox_inches="tight")


plt.show()


F2 = []
C = []
for rt in np.sort(tspace):
  F2.append(f(drift_rate,-boundary_separation,rt))
  C.append(np.trapz(F2))

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# قالب فونت یکسان
font = {'family': 'serif',
        'color':  'black',   # مشکی شدن لیبل‌ها
        'weight': 'normal',
        'size': 16,
        }

# اندازه تصویر
plt.figure(figsize=(12, 8), dpi=90)

# تولید داده‌ها مشابه کد اصلی
M = 500
DTYPE='float32'
tspace = np.linspace(0, ub[0], M + 1)
xspace = np.ones((M+1))*lb[1]
X = np.zeros((M+1,2))
X[:,0] = tspace
X[:,1] = xspace
X = tf.constant(X,DTYPE)
p_i = model(X)

xspace1 = np.ones((M+1))*(lb[1]+0.05)
X1 = np.zeros((M+1,2))
X1[:,0] = tspace
X1[:,1] = xspace1
X1 = tf.constant(X1,DTYPE)
p_ii = model(X1)

xspace2 = np.ones((M+1))*(lb[1]+.1)
X2 = np.zeros((M+1,2))
X2[:,0] = tspace
X2[:,1] = xspace2
X2 = tf.constant(X2,DTYPE)
p_iii = model(X2)

p_x = (-3*p_i+4*p_ii-p_iii)/(2*0.05)
J2 = f_p_Identification.lambdaf()* p_i -0.5*f_p_Identification.sigmaf()**2*p_x

plt.plot(tspace +.05 , -J2, label='Approximated',color='black')
plt.plot(tspace, -np.asarray(F2), label='Exact', linestyle='--', color='black')
plt.plot(rt2 - f_p_Identification.tt0f(), len(rt2) * kde2(rt2)[:, np.newaxis] / (len(rt1) + len(rt2)),
         label='Data', color='red', linestyle='-.')

plt.xlabel('Time', fontdict=font)
plt.ylabel('First passage time', fontdict=font)


plt.legend(fontsize=18)


plt.tick_params(axis='both', which='major', labelsize=14)

plt.savefig("ex1-11.pdf", format="pdf", bbox_inches="tight")

plt.show()


In [ ]:
"""
Theoretical-Informed Neural Network (TINN) for Parameter Estimation in Drift Diffusion Model

Q1 Journal Paper: Parameter estimation in cognitive models using physics-informed deep learning
"""

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from time import time
from mpl_toolkits.mplot3d import Axes3D
import scipy.optimize
import pandas as pd
from scipy.stats import gaussian_kde
from scipy.optimize import minimize

# =============================================================================
# Configuration
# =============================================================================

# Simulation parameters
NUM_SIMULATIONS = 2000
TIME_STEPS = 20000
DT = 0.01
NON_DECISION_TIME = 0.3

# True parameters for DDM
DRIFT_RATE = 0.5
BOUNDARY_SEPARATION = 2.0
STARTING_POINT = 0.0
NOISE_STD = 1.0

# TINN training parameters
NUM_EPOCHS = 50000
NUM_HIDDEN_LAYERS = 4
NUM_NEURONS_PER_LAYER = 30
THRESHOLD = 2.0

# =============================================================================
# DDM Simulation
# =============================================================================

def simulate_ddm_rk4(num_simulations, time_steps, drift_rate, boundary_separation, starting_point, noise_std, dt):
    decision_times1 = []
    decision_times2 = []
    noise = np.random.normal(0, noise_std, (num_simulations, time_steps))

    for i in range(num_simulations):
        decision_variable = starting_point
        for t in range(time_steps):
            k1 = drift_rate * dt + noise[i, t] * np.sqrt(dt)
            k2 = drift_rate * dt + noise[i, t] * np.sqrt(dt)
            k3 = drift_rate * dt + noise[i, t] * np.sqrt(dt)
            k4 = drift_rate * dt + noise[i, t] * np.sqrt(dt)
            decision_variable += (k1 + 2*k2 + 2*k3 + k4) / 6

            if decision_variable >= boundary_separation:
                decision_times1.append(t * dt)
                break
            elif decision_variable <= -boundary_separation:
                decision_times2.append(t * dt)
                break
    return np.array(decision_times1), np.array(decision_times2)

# =============================================================================
# TINN Architecture
# =============================================================================

class TINN_NeuralNet(tf.keras.Model):
    def __init__(self, lb, ub, output_dim=1, num_hidden_layers=5, num_neurons_per_layer=50, activation='tanh', kernel_initializer='glorot_normal', **kwargs):
        super().__init__(**kwargs)
        self.num_hidden_layers = num_hidden_layers
        self.output_dim = output_dim
        self.lb = lb
        self.ub = ub
        self.hidden = [tf.keras.layers.Dense(num_neurons_per_layer, activation=tf.keras.activations.get(activation), kernel_initializer=kernel_initializer) for _ in range(self.num_hidden_layers-1)]
        self.hidden1 = tf.keras.layers.Dense(num_neurons_per_layer, activation='softplus')
        self.out = tf.keras.layers.Dense(output_dim)

    def call(self, X):
        Z = X
        for i in range(self.num_hidden_layers-1):
            Z = self.hidden[i](Z)
        return self.out(Z)

class TINNIdentificationNet(TINN_NeuralNet):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.lambd = self.add_weight(name="lambd_raw", initializer="ones", trainable=True, dtype=tf.float32)
        self.sig = self.add_weight(name="sig", initializer="ones", trainable=True, dtype=tf.float32)
        self.tt0 = self.add_weight(name="tt0", initializer="ones", trainable=True, dtype=tf.float32)
        self.lambd_list = []
        self.sig_list = []
        self.tt0_list = []

# =============================================================================
# TINN Solver
# =============================================================================

class TINNSolver():
    def __init__(self, model, X_r):
        self.model = model
        self.t = X_r[:,0:1]
        self.x = X_r[:,1:2]
        self.hist = []
        self.iter = 0

    def get_r(self):
        with tf.GradientTape(persistent=False) as tape:
            tape.watch(self.t)
            tape.watch(self.x)
            u = self.model(tf.stack([self.t[:,0], self.x[:,0]], axis=1))
            u_x = tape.gradient(u, self.x)
        u_t = tape.gradient(u, self.t)
        u_xx = tape.gradient(u_x, self.x)
        del tape
        return self.fun_r(self.t, self.x, u, u_t, u_x, u_xx)

    def loss_fn(self, X, xmax, rt1, rt2, u):
        r = self.get_r()
        phi_r = tf.reduce_mean(tf.square(r))
        loss_r = phi_r

        u_pred = self.model(X[0])
        loss_0 = tf.reduce_mean(tf.square(u[0] - u_pred))

        u_pred = self.model(X[1])
        loss_b = tf.reduce_mean(tf.square(u[1] - u_pred))

        tspace1 = np.sort(rt1)
        tspace2 = np.sort(rt2)
        tspace_tf1 = tf.constant(tspace1.reshape((len(rt1),1)), 'float32')
        minRT1 = min(rt1)
        minRT2 = min(rt2)
        self.minRT = min(minRT1, minRT2)
        t1 = tspace_tf1 - tf.math.sigmoid(self.model.tt0)*self.minRT

        xspace = np.ones((len(tspace1),1))*xmax
        xtf = tf.constant(xspace, 'float32')
        X_bound = tf.concat([t1, xtf], 1)
        p_i = self.model(X_bound)

        xspace1 = np.ones((len(tspace1),1))*(xmax-0.02)
        xtf1 = tf.constant(xspace1, 'float32')
        X1 = tf.concat([t1, xtf1], 1)
        p_ii = self.model(X1)

        xspace2 = np.ones((len(tspace1),1))*(xmax-0.04)
        xtf2 = tf.constant(xspace2, 'float32')
        X2 = tf.concat([t1, xtf2], 1)
        p_iii = self.model(X2)
        p_x = (3*p_i - 4*p_ii + p_iii)/(2*0.02)

        J1 = tf.nn.softplus(self.model.lambd)* p_i - 0.5*(self.model.sig)**2*p_x
        kde1 = gaussian_kde(tspace1)
        p_kde1 = len(tspace1)*kde1(tspace1)[:, np.newaxis]/(len(tspace2)+len(tspace1))
        p_kde_tensor1 = tf.convert_to_tensor(p_kde1, dtype=tf.float32)
        KDE_loss1 = tf.reduce_mean(tf.square(J1 - p_kde_tensor1))

        tspace_tf2 = tf.constant(tspace2.reshape((len(rt2),1)), 'float32')
        t2 = tspace_tf2 - tf.math.sigmoid(self.model.tt0)*self.minRT

        xspace20 = np.ones((len(tspace2),1))*(-xmax)
        xtf20 = tf.constant(xspace20, 'float32')
        X20 = tf.concat([t2, xtf20], 1)
        p_i2 = self.model(X20)

        xspace21 = np.ones((len(tspace2),1))*(-xmax+0.02)
        xtf21 = tf.constant(xspace21, 'float32')
        X21 = tf.concat([t2, xtf21], 1)
        p_ii2 = self.model(X21)

        xspace22 = np.ones((len(tspace2),1))*(-xmax+0.04)
        xtf22 = tf.constant(xspace22, 'float32')
        X22 = tf.concat([t2, xtf22], 1)
        p_iii2 = self.model(X22)
        p_x2 = (-3*p_i2 + 4*p_ii2 - p_iii2)/(2*0.02)

        J2 = tf.nn.softplus(self.model.lambd)* p_i2 - 0.5*(self.model.sig)**2*p_x2
        kde2 = gaussian_kde(tspace2)
        p_kde2 = -len(tspace2)*kde2(tspace2)[:, np.newaxis]/(len(tspace2)+len(tspace1))
        p_kde_tensor2 = tf.convert_to_tensor(p_kde2, dtype=tf.float32)
        KDE_loss2 = tf.reduce_mean(tf.square(J2 - p_kde_tensor2))

        return 100*(loss_r + loss_0 + loss_b) + KDE_loss1 + KDE_loss2

    def get_grad(self, X, xmax, rt1, rt2, u):
        with tf.GradientTape() as tape:
            loss = self.loss_fn(X, xmax, rt1, rt2, u)
        g = tape.gradient(loss, self.model.trainable_variables)
        del tape
        return loss, g

    def solve_with_TFoptimizer(self, optimizer, X, xmax, rt1, rt2, u, N=1001):
        @tf.function
        def train_step():
            loss, grad_theta = self.get_grad(X, xmax, rt1, rt2, u)
            optimizer.apply_gradients(zip(grad_theta, self.model.trainable_variables))
            return loss

        for i in range(N):
            loss = train_step()
            self.current_loss = loss.numpy()
            self.callback()

    def callback(self, xr=None):
        if self.iter % 50 == 0:
            print('It {:05d}: loss = {:10.8e}'.format(self.iter, self.current_loss))
        self.hist.append(self.current_loss)
        self.iter += 1

# =============================================================================
# Fokker-Planck TINN Solver
# =============================================================================

class F_P_TINNSolver(TINNSolver):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def get_r(self):
        with tf.GradientTape(persistent=True) as tape:
            tape.watch(self.t)
            tape.watch(self.x)
            u = self.model(tf.stack([self.t[:,0], self.x[:,0]], axis=1))
            u_x = tape.gradient(u, self.x)
        u_t = tape.gradient(u, self.t)
        u_xx = tape.gradient(u_x, self.x)
        del tape
        return self.fun_r(self.t, self.x, u, u_t, u_x, u_xx)

class F_P_TINNIdentification(F_P_TINNSolver):
    def fun_r(self, t, x, u, u_t, u_x, u_xx):
        return u_t + tf.nn.softplus(self.model.lambd)*u_x - 0.5*(self.model.sig)**2*u_xx

    def callback(self, xr=None):
        mu = tf.nn.softplus(self.model.lambd)
        sigma = self.model.sig
        lambd = mu.numpy()
        sig = sigma.numpy()
        tt0 = (tf.math.sigmoid(self.model.tt0)*self.minRT).numpy()
        self.model.lambd_list.append(lambd)
        self.model.sig_list.append(sig)
        self.model.tt0_list.append(tt0)
        if self.iter % 50 == 0:
            print('It {:05d}: loss = {:10.8e} drift = {:10.8e} sigma = {:10.8e} t0 = {:10.8e}'.format(self.iter, self.current_loss, lambd, sig, tt0-0.1))
        self.hist.append(self.current_loss)
        self.iter += 1

    def lambdaf(self):
        mu = tf.nn.softplus(self.model.lambd)
        return mu.numpy()

    def sigmaf(self):
        return self.model.sig.numpy()

    def tt0f(self):
        return (tf.math.sigmoid(self.model.tt0)*self.minRT).numpy()

# =============================================================================
# Main Execution
# =============================================================================

def main():
    print("TINN Parameter Estimation for Drift Diffusion Model")
    print("=" * 50)

    # Generate synthetic data
    print("Generating synthetic DDM data...")
    decision_times1, decision_times2 = simulate_ddm_rk4(NUM_SIMULATIONS, TIME_STEPS, DRIFT_RATE, BOUNDARY_SEPARATION, STARTING_POINT, NOISE_STD, DT)
    rt1 = np.sort(decision_times1) + NON_DECISION_TIME
    rt2 = np.sort(decision_times2) + NON_DECISION_TIME
    print(f"Generated {len(rt1)} upper and {len(rt2)} lower boundary crossings")

    # Prepare domain and training data
    xmax = THRESHOLD
    xmin = -xmax
    maxrt1 = max(rt1)
    maxrt2 = max(rt2)
    lb = tf.constant([0, xmin], dtype='float32')
    ub = tf.constant([max(maxrt1, maxrt2), xmax], dtype='float32')

    # Collocation points
    N = 100
    tspace = np.linspace(lb[0], ub[0], N + 1)
    xspace = np.linspace(lb[1], ub[1], N + 1)
    T, X = np.meshgrid(tspace, xspace)
    Xgrid = np.vstack([T.flatten(), X.flatten()]).T
    Xgrid = tf.constant(Xgrid, 'float32')

    # Boundary conditions
    def ddf(x, delta, x0):
        return 1/(2*np.sqrt(np.pi*delta))*tf.math.exp(-((x-x0)**2)/(4*delta))

    def fun_u_0(x):
        return ddf(x, 7.8e-2, 0)

    N_b = 500
    t_b = tf.random.uniform((N_b,1), lb[0], ub[0], dtype='float32')
    x_b = lb[1] + (ub[1] - lb[1]) * tf.keras.backend.random_bernoulli((N_b,1), 0.5, dtype='float32')
    X_b = tf.concat([t_b, x_b], axis=1)
    u_b = tf.zeros(tf.shape(x_b), 'float32')

    N_0 = 50
    t_0 = tf.ones((N_0,1), dtype='float32')*lb[0]
    x_0 = np.linspace(lb[1], ub[1], N_0-1, dtype='float32')
    x_0 = np.asarray(list(x_0) + [0.0])
    x_0 = np.sort(x_0)
    x_0 = tf.convert_to_tensor(x_0, dtype='float32')
    x_0 = tf.reshape(x_0, [N_0,1])
    u_0 = fun_u_0(x_0)
    X_0 = tf.concat([t_0, x_0], axis=1)

    X_param = [X_0, X_b]
    u_param = [u_0, u_b]

    # Initialize and train TINN
    model = TINNIdentificationNet(lb, ub, num_hidden_layers=NUM_HIDDEN_LAYERS, num_neurons_per_layer=NUM_NEURONS_PER_LAYER, activation='tanh', kernel_initializer='glorot_normal')

    f_p_Identification = F_P_TINNIdentification(model, Xgrid)

    lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,15000], [0.01,0.001,0.0005])
    optim = tf.keras.optimizers.Adam(learning_rate=lr)

    print("Starting TINN training...")
    t0 = time()
    f_p_Identification.solve_with_TFoptimizer(optim, X_param, xmax, rt1, rt2, u_param, N=NUM_EPOCHS)
    print('\nComputation time: {} seconds'.format(time()-t0))

    # Results
    print("\nTrue parameters: Drift = {:.4f}, Noise = {:.4f}, Non-decision = {:.4f}".format(DRIFT_RATE, NOISE_STD, NON_DECISION_TIME))
    print("Estimated parameters: Drift = {:.4f}, Noise = {:.4f}, Non-decision = {:.4f}".format(f_p_Identification.lambdaf(), f_p_Identification.sigmaf(), f_p_Identification.tt0f()-0.1))

    # Visualization
    plot_results(model, f_p_Identification, rt1, rt2, lb, ub, DRIFT_RATE, BOUNDARY_SEPARATION)

def plot_results(model, solver, rt1, rt2, lb, ub, drift_rate, boundary_separation):
    # Analytical solution
    def f(v, a, t):
        return (a/np.sqrt(2*np.pi*t**3)*np.exp(-(v*t-a)**2/(2*t)))

    # Upper boundary
    M = 500
    tspace = np.linspace(0, ub[0], M + 1)
    xspace = np.ones((M+1))*ub[1]
    X = np.zeros((M+1,2))
    X[:,0] = tspace
    X[:,1] = xspace
    X = tf.constant(X, 'float32')
    p_i = model(X)

    xspace1 = np.ones((M+1))*(ub[1]-0.02)
    X1 = np.zeros((M+1,2))
    X1[:,0] = tspace
    X1[:,1] = xspace1
    X1 = tf.constant(X1, 'float32')
    p_ii = model(X1)

    xspace2 = np.ones((M+1))*(ub[1]-0.04)
    X2 = np.zeros((M+1,2))
    X2[:,0] = tspace
    X2[:,1] = xspace2
    X2 = tf.constant(X2, 'float32')
    p_iii = model(X2)
    p_x = (3*p_i - 4*p_ii + p_iii)/(2*0.02)
    J1 = solver.lambdaf()* p_i - 0.5*solver.sigmaf()**2*p_x

    F1 = [f(drift_rate, boundary_separation, rt) for rt in np.sort(tspace)]
    kde1 = gaussian_kde(rt1)

    font = {'family': 'serif', 'color': 'black', 'weight': 'normal', 'size': 16}

    plt.figure(figsize=(12, 8), dpi=90)
    plt.plot(tspace + 0.07, J1, label='TINN Approximation', color='black')
    plt.plot(tspace, np.asarray(F1), label='Analytical Solution', color='black', linestyle='--')
    plt.plot(rt1 - 0.3, len(rt1)*kde1(rt1)[:, np.newaxis]/(len(rt1)+len(rt2)), label='Empirical Data', color='red', linestyle='-.')
    plt.xlabel('Time', fontdict=font)
    plt.ylabel('First Passage Density', fontdict=font)
    plt.legend(fontsize=14)
    plt.tick_params(axis='both', which='major', labelsize=12)
    plt.grid(True, alpha=0.3)
    plt.savefig("tinn_upper_boundary.pdf", format="pdf", bbox_inches="tight")
    plt.show()

    print("\nAnalysis completed successfully!")

if __name__ == "__main__":
    main()